In [2]:
import json

In [3]:
# load a json file
with open(r"C:\Users\David gathara marigi\Downloads\bitcoin_price.json") as f:
    data = json.load(f)

In [4]:
# save the data into sqlite database
import sqlite3

# connect to (or create) the SQLite database
conn = sqlite3.connect("bitcoin_data.db")
cursor = conn.cursor()

# create a table to store bitcoin pricing data
cursor.execute("""
    CREATE TABLE IF NOT EXISTS bitcoin_data (
        symbol TEXT,
        time INTEGER,
        open REAL,
        close REAL,
        high REAL,
        low REAL,
        volume REAL
    )
""")

# insert the data from the JSON loaded 'data' variable
cursor.executemany("""
    INSERT INTO bitcoin_data (symbol, time, open, close, high, low, volume)
    VALUES (?, ?, ?, ?, ?, ?, ?)
""", [(item["symbol"], item["time"], item["open"], item["close"], item["high"], item["low"], item["volume"]) for item in data])

# save (commit) the changes and close the connection
conn.commit()
conn.close()


In [5]:
import datetime
import sqlite3
conn = sqlite3.connect("bitcoin_data.db")
cursor = conn.cursor()

# Query 1: Get the earliest and latest timestamps
result = cursor.execute("SELECT MIN(time) as earliest, MAX(time) as latest FROM bitcoin_data").fetchone()
print("Earliest timestamp:", result[0])
print("Latest timestamp:", result[1])

# Optionally, convert timestamps from milliseconds to human-readable dates
earliest_date = datetime.datetime.fromtimestamp(result[0] / 1000)
latest_date = datetime.datetime.fromtimestamp(result[1] / 1000)
print("Earliest date:", earliest_date)
print("Latest date:", latest_date)

# Query 2: Get maximum high price for each year grouped and sorted by year
query = """
SELECT 
    strftime('%Y', datetime(time/1000, 'unixepoch')) AS year, 
    MAX(high) AS max_price
FROM bitcoin_data
GROUP BY year
ORDER BY year;
"""
yearly_max = cursor.execute(query).fetchall()
print("Maximum price for each year:")
for year, price in yearly_max:
    print(f"{year}: {price}")

Earliest timestamp: 1364688000000
Latest timestamp: 1606521600000
Earliest date: 2013-03-31 03:00:00
Latest date: 2020-11-28 03:00:00
Maximum price for each year:
2013: 1175.0
2014: 995.0
2015: 504.0
2016: 983.0
2017: 19891.0
2018: 17252.0
2019: 13764.0
2020: 19497.0


In [6]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///chinook.db")
conn_engine = engine.connect()
print("Connected to chinook.db")

Connected to chinook.db


In [7]:
# Ensure you are using the correct Chinook database that includes the Genre and Track tables.
# If you haven't already, download the Chinook sample database from:
# https://github.com/lerocha/chinook-database and set the correct file path for chinook.db.
query_genre = """
SELECT 
    g.Name AS genre, 
    AVG(t.Milliseconds) AS avg_length
FROM Genre g
JOIN Track t ON g.GenreId = t.GenreId
GROUP BY g.Name
ORDER BY avg_length DESC;
"""
from sqlalchemy import text
try:
    result_genre = conn_engine.execute(text(query_genre)).fetchall()
    for genre, avg_length in result_genre:
        print(f"Genre: {genre}, Average Length: {avg_length:.2f} milliseconds")
except Exception as e:
    print("Error executing query. Please ensure that your chinook.db file contains the Genre and Track tables.")
    print(e)

Error executing query. Please ensure that your chinook.db file contains the Genre and Track tables.
(sqlite3.OperationalError) no such table: Genre
[SQL: 
SELECT 
    g.Name AS genre, 
    AVG(t.Milliseconds) AS avg_length
FROM Genre g
JOIN Track t ON g.GenreId = t.GenreId
GROUP BY g.Name
ORDER BY avg_length DESC;
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
